In [16]:
from bs4 import BeautifulSoup
from IPython.display import clear_output, display
import requests
import csv
import re
import datetime

In [29]:
def obtener_avisos():
    url = "https://www.yapo.cl/chile/comprar?ca=15_s&l=0&cmn="
    request = requests.get(url)
    soup = BeautifulSoup(request.text, "html.parser")
    ultima_pagina_link = soup.find('span',{'class':'nohistory FloatRight'}).find('a').get('href')
    ultima_pagina = int(ultima_pagina_link.split("o=",1)[1])

    inicio = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    #ultima_pagina = 2
    total_data = []
    variables = []
    n = 0
    
    for i in range(1,ultima_pagina+1):
        url_page = url + "&st=a&o=" + str(i)
        #print(url_page)
        data = requests.get(url_page)
        soup_inner = BeautifulSoup(data.text, "html.parser")
        
        for row in soup_inner.find_all('tr', {'class':'ad listing_thumbs'}):
            a = {}
            #col = row.find_all('td')
            a['data_scraping'] = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")    
        
            try: a['id_anuncio'] = row.get('id').encode("utf-8") 
            except: variables.append('id_anuncio')
            
            try: a['enlace'] = str(row.find_all('td')[0].find('a').get('href').encode("utf-8")).replace("b'","").replace("'","")
            except: variables.append('enlace')
            
            try: a['titulo'] = row.find_all('td')[2].find('a').text.encode("utf-8")
            except: variables.append('titulo')
            
            try: a['precio'] = re.sub(r'(^[ \t]+|[ \t]+(?=:))', '', row.find_all('td')[2].find('span', {'class':'price'}).text, flags=re.M).replace("\n", "")
            except: variables.append('precio')
            
            try: a['precio2'] = re.sub(r'(^[ \t]+|[ \t]+(?=:))', '', row.find_all('td')[2].find('span', {'class':'convertedPrice'}).text, flags=re.M).replace("\n", "")
            except: variables.append('precio2')
            
            datos_iconos = row.find_all('td')[2]
            
            try: a['banos'] = str(datos_iconos).split('fa-bath',1)[1].split('</span>')[0].split('-text">')[1]
            except: variables.append('banos')
            
            try: a['dormitorios'] = str(datos_iconos).split('fa-bed',1)[1].split('</span>')[0].split('-text">')[1]
            except: variables.append('dormitorios')
            
            try: a['superficie'] = BeautifulSoup(str(datos_iconos).split('fa-expand',1)[1], "html.parser").text.replace(' icons__element-icon">','').replace("\n", "")
            except: variables.append('superficie')
            
            datos_ubicacion = row.find_all('td')[3]
            
            try: a['region'] = str(datos_ubicacion).split('class="region">')[1].split('</span>')[0]
            except: variables.append('region')
            
            try: a['comuna'] = str(datos_ubicacion).split('class="commune">')[1].split('</span>')[0]
            except: variables.append('comuna')
            
            try: a['company_ad'] = str(datos_ubicacion).split('title="Aviso profesional">')[1].split('</span>')[0]
            except: variables.append('company_ad')
            
            try: a['ubicacion'] = datos_ubicacion.find('i').get('class')[1]
            except:variables.append('ubicacion')
            
            each_sales_request = requests.get(a['enlace'])
            each_sales_soup = BeautifulSoup(each_sales_request.text, "html.parser")
        
            datos_aviso = each_sales_soup.find('section',{'class':'da-wrapper'})
        
            try: a['datetime_data'] = datos_aviso.find('article').find_all('div')[0].get('data-datetime')
            except: variables.append('datetime_data')
            
            try: a['vendedor'] = datos_aviso.find('seller-info').get('username')
            except: variables.append('vendedor')
        
            try: a['geo_data'] = str(datos_aviso.find('div',{'id':'modal_map_wrap'})).split(']')[0].replace('"[','').split(' data-location=')[1].replace(" ","")
            except: variables.append('geo_data')
        
            try: a['direccion'] = str(datos_aviso.find('h2')).replace('<h2>','').replace('</h2>','')
            except: variables.append('direccion')
        
            try: a['imagen'] = datos_aviso.find('span',{'id':'display_image'}).find('img').get('src')
            except: variables.append('imagen')
            
            try: a['descripcion'] = datos_aviso.find('p',{'itemprop':'description'}).text
            except: variables.append('descripcion')
        
            try: 
                detalles_aviso = datos_aviso.find_all('div',{'class':'details'})
                for tr in detalles_aviso:
                    datos = tr.find_all('tr')
                    for z in range(0,len(datos)):
                        k = 0
                        limit = len(datos)
                        while k < limit: 
                            #print (str(datos[z].find('th')) + " : " + str(datos[z].find('td')))
                            item = datos[z].find('th').text.encode('utf-8')
                            item = re.sub(r"b'", "", str(item))
                            item = re.sub(r"'", "", str(item))
                            item = re.sub(r"\\xc3\\xb3", "o", str(item))
                            if item not in variables:
                                variables.append(item)
                            try: itemvalue = datos[z].find('td').text.encode('utf-8')
                            except: itemvalue = ""
                            a[item] = itemvalue
                            k += 1
            except: pass
                        
            clear_output(wait=True)
            n = n+1
            print("Resultados encontrados! Cantidad de paginas a scrapear: " + str(ultima_pagina))
            print("Inicio: " + inicio)
            print("Scrapeando página: " + url_page)
            print("Avisos scrapeados: " + str(n) + " de " + str(ultima_pagina*50) + " aprox. (" + str(n*100/(ultima_pagina*50)) + ")%")
            total_data.append(a)

        
    for i in range(0,len(total_data)):
        for n in range(0,len(variables)):
            if variables[n] not in total_data[i]:
                total_data[i][variables[n]] = ""
    
    keys = total_data[0].keys()
    now = datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S")
    csvname = "data "
    with open(str(csvname)+str(now)+'.csv','w', newline='', encoding="utf-8-sig", errors='surrogatepass') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(total_data)
    print("Terminado!")
    print("Término: " + datetime.datetime.now().strftime("%Y-%m-%d %H-%M-%S"))
    print("Los resultados pueden ser encontrados en el archivo "+str(csvname)+str(now)+".csv")

In [31]:
obtener_avisos()

Resultados encontrados! Cantidad de paginas a scrapear: 997
Inicio: 2021-05-02 14-05-37
Scrapeando página: https://www.yapo.cl/chile/comprar?ca=15_s&l=0&cmn=&st=a&o=1
Avisos scrapeados: 8 de 49850 aprox. (0.0160481444332999)%


KeyboardInterrupt: 